In [ ]:
# Add project src to path.
import set_path

import numpy as np
import logging

import src.region_growing as growing
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
from src.labels import Labels
from src.pipeline import Pipeline

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

In [ ]:
tilecode = '2623_9621'
# tilecode = '2397_9705'

in_folder = '../../datasets/Weesp_train/new_10_classes/'
in_prefix = 'processed_'
out_folder = '../../datasets/Weesp_train/new_10_classes_refined/'
out_prefix = 'refined_'

in_file = f'{in_folder}{in_prefix}{tilecode}.laz'
out_file = f'{out_folder}{out_prefix}{tilecode}.laz'

target_labels = growing.RefineGround.TARGET_LABELS
exclude_labels = (Labels.NOISE,)

ahn_data_folder = '../../datasets/Weesp/ahn4_tiles/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

In [ ]:
params = {'top': 1., 'grid_size': 0.4, 'min_comp_size': 100}

refiner = growing.RefineGround(target_labels, npz_reader, params)

---
"Manual" processing of the tile.

In [ ]:
pointcloud = las_utils.read_las(in_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

In [ ]:
mask = np.ones((len(points),), dtype=bool)
for l in exclude_labels:
    mask = mask & (labels != l)

In [ ]:
ref_mask = refiner.get_label_mask(points, labels, mask, tilecode)

In [ ]:
for label, mask in zip(target_labels, ref_mask):
    labels[mask] = label

In [ ]:
las_utils.label_and_save_las(pointcloud, labels, out_file)

In [ ]:
type(ref_mask) == list

---
Add it to the pipeline.

In [ ]:
# Set-up region growing based on PCL pipeline.
process_sequence = (refiner,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

In [ ]:
pipeline.process_file(in_file, out_file=out_file)

In [ ]:
log_file = 'refine.log'
# Re-set logging to suppress console output.
log_utils.reset_logger()
# Only ERROR messages will be printed to console.
log_utils.add_console_logger(level=logging.ERROR)
# All messages will be printed to a file. Use `clear_log` flag to clear the log file, if desired.
log_utils.add_file_logger(log_file, clear_log=True)

In [ ]:
pipeline.process_folder(in_folder=in_folder, out_folder=out_folder, in_prefix=in_prefix, out_prefix=out_prefix)

---

In [ ]:
tilecode = '2630_9603'
in_file = f'{in_folder}{in_prefix}{tilecode}.laz'
out_file = f'{out_folder}{out_prefix}{tilecode}.laz'

In [ ]:
import numpy as np
import laspy

def clean_las(in_file, out_file):
    pc = laspy.read(in_file)
    points = np.vstack((pc.x, pc.y, pc.z)).T
    rgb = np.vstack((pc.red, pc.green, pc.blue)).T
    
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.add_extra_dim(laspy.ExtraBytesParams(name="label", type=np.uint8,
                                                description="Labels"))
    header.offsets = pc.header.offsets
    header.scales = pc.header.scales

    clean_pc = laspy.LasData(header)
    clean_pc.x = points[:, 0]
    clean_pc.y = points[:, 1]
    clean_pc.z = points[:, 2]
    max_c = np.max(rgb)
    if max_c <= 1.:
        rgb = (rgb * 2**8).astype('uint8')
    elif max_c <= 2**8:
        pass
    elif max_c <= 2**16:
        rgb = (rgb / 2**8).astype('uint8')
    else:
        print(f'Something weird is going on in {in_file}.')
    clean_pc.red = rgb[:, 0]
    clean_pc.green = rgb[:, 1]
    clean_pc.blue = rgb[:, 2]
    clean_pc.intensity = pc.intensity
    clean_pc.gps_time = pc.gps_time
    clean_pc.return_number = pc.return_number
    clean_pc.number_of_returns = pc.number_of_returns
    clean_pc.label = pc.label.astype('uint8')
    clean_pc.write(out_file)

In [ ]:
tilecode = '2630_9603'
in_folder = '../../datasets/Weesp_train/new_10_classes/'
in_prefix = 'processed_'
out_folder = '../../datasets/Weesp_train/new_10_classes_refined/'
out_prefix = 'refined_'

# in_file = f'{out_folder}{out_prefix}{tilecode}.laz'
in_file = f'{in_folder}{in_prefix}{tilecode}.laz'
out_file = '../../datasets/Weesp_train/processed_2630_9603.laz'

In [ ]:
import pathlib
in_folder = '../../datasets/Weesp_train/new_10_classes_refined/'
in_prefix = 'refined_'

files = list(pathlib.Path(in_folder).glob(f'{in_prefix}*.laz'))
for f in files:
    tilecode = las_utils.get_tilecode_from_filename(f.as_posix())
    las = laspy.read(f)
    print(f'{tilecode}: {las.label.dtype}')